In [3]:
import numpy as np
import damask
import shutil
import os
from math import * 
os.getcwd()
print("The current directory is: ")
print(os.getcwd())
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
print("Moving up to the parent directory")
print(os.getcwd())

The current directory is: 
c:\Users\springnuance\Desktop\Crystal-Plasticity-Journal-Project\damask3
Moving up to the parent directory
c:\Users\springnuance\Desktop\Crystal-Plasticity-Journal-Project


In [6]:
material = "RVE_1_40_D"

CPLaws = ["PH", "DB"]

loadings = [
    "linear_uniaxial_RD",
    "linear_uniaxial_TD",
    "nonlinear_biaxial_RD",
    "nonlinear_biaxial_TD",
    "nonlinear_planestrain_RD",
    "nonlinear_planestrain_TD",
    "nonlinear_uniaxial_RD",
    "nonlinear_uniaxial_TD",
]

In each source/material_name folder, there must exists these files:

- debug.yaml
- numerics.yaml
- material.geom file extracted from Dream3D
- euler_angles.txt extracted from end of material file from Dream3D
- preprocessing.sh
- postprocessing.sh

In [20]:
for loading in loadings:
    for CPLaw in CPLaws:
        dst_dir = f'damask3/templates/{material}/{CPLaw}/{loading}'
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

### Copying shell scripts

In [11]:
for loading in loadings:
    for CPLaw in CPLaws:
        src_file = f'damask3/source/{material}/preprocessing.sh'
        shutil.copy2(src_file, dst_dir)
        src_file = f'damask3/source/{material}/postprocessing.sh'
        shutil.copy2(src_file, dst_dir)

### Copying debug.yaml and numerics.yaml

In [14]:
numerics = damask.Config(f"""
grid:
  itmin: 2
  itmax: 250
  maxCutBack: 5
""")

for loading in loadings:
    for CPLaw in CPLaws:
      numerics.save(f"damask3/templates/{material}/{CPLaw}/{loading}/numerics.yaml",default_flow_style = False)

debug = damask.Config(f"""
  phase: [basic, extensive, selective]
  materialpoint: [basic, extensive, selective]

  # options for selective debugging
  element: 1
  integrationpoint: 1
  constituent: 1

  # solver-specific
  mesh: [PETSc]
  grid: [basic, rotation, PETSc]
  Marc: [basic]
""")

for loading in loadings:
    for CPLaw in CPLaws:
      debug.save(f"damask3/templates/{material}/{CPLaw}/{loading}/debug.yaml",default_flow_style = False)

### Converting Dream3D geom file to .vti file and save it

In [16]:
geometry = damask.Grid.load_ASCII(f'damask3/source/{material}/RVE_1_40_D.geom')

print(geometry)
print(geometry.material)
for loading in loadings:
    for CPLaw in CPLaws:
        geometry.save(f"damask3/templates/{material}/{CPLaw}/{loading}/RVE_1_40_D.vti")

C:\Users\springnuance\AppData\Local\Temp\ipykernel_9464\1034445354.py:1: DeprecationWarning: Support for ASCII-based geom format will be removed in DAMASK 3.0.0
  geometry = damask.Grid.load_ASCII(f'damask3/source/{material}//RVE_1_40_D.geom')


cells:  40 × 40 × 40
size:   0.003 × 0.003 × 0.003 m³
origin: 0.0   0.0   0.0 m
# materials: 3165
[[[ 482  482  482 ... 1605 1605  162]
  [1121 1121 1121 ... 1605  162  162]
  [ 162  162 1121 ... 1605  162  162]
  ...
  [ 687 2575 2575 ... 2516 2516 2516]
  [2863 2863 2575 ... 2516 2516 2516]
  [2863 2863 1490 ... 2516 2516 2516]]

 [[ 482  482  482 ... 1605 1605  162]
  [ 162  482  482 ... 1605  162  162]
  [ 162  482  482 ... 1605  162  162]
  ...
  [ 687  687 2575 ... 2694 2694 2694]
  [1203 1203 1203 ... 2694 2694 2694]
  [1203 1203 1203 ... 2694 2694 2694]]

 [[ 162  482  482 ... 1605 1605  162]
  [ 162  482  482 ... 1605  162  162]
  [ 162  482  482 ... 1605  162  162]
  ...
  [ 687  506  506 ... 1164 2694 2349]
  [ 506  506 1768 ... 2694 2694 2349]
  [1203 1203 1768 ... 2233 1902 1902]]

 ...

 [[1285  761  761 ... 1698 2789 2789]
  [1285  761  761 ... 1698 1698 2789]
  [2985  761  761 ... 1698 1948 1948]
  ...
  [2221 2221 2575 ... 2773 2773 2177]
  [2789 1285 1285 ... 2773  28